In [1]:
!pip install scikit-image

In [2]:
from IPython import get_ipython
from IPython.display import display
# ---- srgan_training_eval.py ----
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.utils import save_image
from PIL import Image
from tqdm import tqdm
import os
import requests, zipfile, io
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np
import glob

In [3]:
# --- Download DIV2K Dataset (HR + LR) ---
def download_div2k(root_dir):
    os.makedirs(root_dir, exist_ok=True)

    hr_url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip"
    lr_url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip"

    hr_zip_path = os.path.join(root_dir, "DIV2K_train_HR.zip")
    lr_zip_path = os.path.join(root_dir, "DIV2K_train_LR_bicubic_X4.zip")

    if not os.path.exists(os.path.join(root_dir, "DIV2K_train_HR")):
        print("Downloading DIV2K HR dataset...")
        r = requests.get(hr_url)
        with open(hr_zip_path, 'wb') as f:
            f.write(r.content)
        with zipfile.ZipFile(hr_zip_path, 'r') as zip_ref:
            zip_ref.extractall(root_dir)

    if not os.path.exists(os.path.join(root_dir, "DIV2K_train_LR_bicubic")):
        print("Downloading DIV2K LR dataset...")
        r = requests.get(lr_url)
        with open(lr_zip_path, 'wb') as f:
            f.write(r.content)
        with zipfile.ZipFile(lr_zip_path, 'r') as zip_ref:
            zip_ref.extractall(root_dir)

In [4]:
# --- Custom Dataset Loader ---
class DIV2KDataset(Dataset):
    def __init__(self, root_dir, crop_size=256): # Added crop_size parameter
        self.hr_dir = os.path.join(root_dir, 'DIV2K_train_HR')
        self.lr_dir = os.path.join(root_dir, 'DIV2K_train_LR_bicubic', 'X4')
        self.hr_files = sorted(glob.glob(os.path.join(self.hr_dir, '*.png')))
        self.lr_files = sorted(glob.glob(os.path.join(self.lr_dir, '*.png')))

        # Added resizing and center cropping transforms
        self.hr_transform = transforms.Compose([
            transforms.Resize(crop_size),
            transforms.CenterCrop(crop_size),
            transforms.ToTensor()
        ])
        self.lr_transform = transforms.Compose([
            transforms.Resize(crop_size // 4), # Resize LR to 1/4th of HR size
            transforms.CenterCrop(crop_size // 4),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.hr_files)

    def __getitem__(self, idx):
        hr_img = Image.open(self.hr_files[idx]).convert('RGB')
        lr_img = Image.open(self.lr_files[idx]).convert('RGB')
        return self.lr_transform(lr_img), self.hr_transform(hr_img)

In [5]:
# --- ESRGAN Generator (RRDBNet) ---
class ResidualDenseBlock(nn.Module):
    def __init__(self, nf=64, gc=32):
        super().__init__()
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return x + 0.2 * x5

class RRDB(nn.Module):
    def __init__(self, nf, gc=32):
        super().__init__()
        self.rdb1 = ResidualDenseBlock(nf, gc)
        self.rdb2 = ResidualDenseBlock(nf, gc)
        self.rdb3 = ResidualDenseBlock(nf, gc)
    def forward(self, x):
        return x + 0.2 * self.rdb3(self.rdb2(self.rdb1(x)))

class RRDBNet(nn.Module):
    def __init__(self, in_nc=3, out_nc=3, nf=64, nb=23, gc=32):
        super().__init__()
        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1)
        self.RRDB_trunk = nn.Sequential(*[RRDB(nf, gc) for _ in range(nb)])
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1)
        self.upsample = nn.Sequential(
            nn.Conv2d(nf, nf * 4, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf, nf * 4, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, True)
        )
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1)
    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea = fea + trunk
        fea = self.upsample(fea)
        return self.conv_last(fea)

In [10]:
# ---- Training and Evaluation Loop ----
def train(model, dataloader, device, epochs=50, lr=1e-4):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.L1Loss()
    model.train()

    for epoch in range(epochs):
        epoch_loss = 0
        with tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
            for lr_img, hr_img in pbar:
                lr_img, hr_img = lr_img.to(device), hr_img.to(device)
                optimizer.zero_grad()
                sr_img = model(lr_img)
                loss = criterion(sr_img, hr_img)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                pbar.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1} Average Loss: {epoch_loss/len(dataloader):.4f}")

        # --- Save images and calculate metrics after each epoch ---
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            sample_lr, sample_hr = next(iter(dataloader)) # Get a sample from the dataloader
            sample_lr, sample_hr = sample_lr.to(device), sample_hr.to(device)
            sample_sr = model(sample_lr).clamp(0, 1)

            # Resize lr_img to the size of sr_img for visual comparison
            lr_img_resized = F.interpolate(sample_lr, size=sample_sr.shape[-2:], mode='bilinear', align_corners=False)

            # Save the resized input (low-resolution) image
            save_image(lr_img_resized[0], f"lr_epoch_{epoch+1}.png")

            # Save the improved (super-resolved) image
            save_image(sample_sr[0], f"sr_epoch_{epoch+1}.png")

            # Calculate PSNR and SSIM for the super-resolved image
            s_np = sample_sr[0].cpu().permute(1, 2, 0).numpy()
            h_np = sample_hr[0].cpu().permute(1, 2, 0).numpy()
            psnr_score = psnr(h_np, s_np, data_range=1.0)
            ssim_score = ssim(h_np, s_np, channel_axis=2, data_range=1.0)
            print(f"Epoch {epoch+1} - PSNR: {psnr_score:.2f}, SSIM: {ssim_score:.4f}")
        model.train() # Set model back to training mode


In [7]:
# ---- Evaluation Metrics ----
def evaluate(model, dataloader, device):
    model.eval()
    psnr_scores, ssim_scores = [], []
    with torch.no_grad():
        for lr_img, hr_img in tqdm(dataloader, desc="Evaluating"):
            lr_img, hr_img = lr_img.to(device), hr_img.to(device)
            sr_img = model(lr_img).clamp(0, 1)
            for s, h in zip(sr_img, hr_img):
                s_np = s.cpu().permute(1,2,0).numpy()
                h_np = h.cpu().permute(1,2,0).numpy()
                psnr_scores.append(psnr(h_np, s_np, data_range=1.0))
                ssim_scores.append(ssim(h_np, s_np, channel_axis=2, data_range=1.0))
    print(f"Average PSNR: {np.mean(psnr_scores):.2f}, SSIM: {np.mean(ssim_scores):.4f}")

In [8]:
download_div2k("data")

In [11]:
# ---- Run in Colab ----
if __name__ == '__main__':
    dataset = DIV2KDataset("data")
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RRDBNet()
    train(model, dataloader, device, epochs=50)
    evaluate(model, dataloader, device)

Epoch 1/50: 100%|██████████| 100/100 [01:42<00:00,  1.02s/it, loss=0.0794]

Epoch 1 Average Loss: 0.1161


Epoch 1 - PSNR: 19.60, SSIM: 0.4473


Epoch 2/50: 100%|██████████| 100/100 [01:42<00:00,  1.03s/it, loss=0.0562]

Epoch 2 Average Loss: 0.0629


...

Epoch 50/50: 100%|██████████| 100/100 [01:42<00:00,  1.02s/it, loss=0.0482]

Epoch 50 Average Loss: 0.0441


Epoch 50 - PSNR: 29.06, SSIM: 0.8285


Evaluating: 100%|██████████| 100/100 [01:30<00:00,  1.10it/s]

Average PSNR: 23.89, SSIM: 0.7134
